In [41]:
# for debugging
import sys, importlib
if 'api' in sys.modules:
    importlib.reload(sys.modules['api'])
from api import HEADERS
import requests

# real
from api import search_posts, get_comments

In [2]:
search_res = search_posts("Coldplay", "stanford")

- Query: https://oauth.reddit.com/r/Coldplay/search?q=stanford&sort_by=new&limit=100&restrict_sr=true
- Query: https://oauth.reddit.com/r/Coldplay/search?q=stanford&sort_by=new&limit=100&restrict_sr=true&after=t3_1kwus1m
Submissions query for subreddit_name='Coldplay', query_term='stanford' yielded 122 submissions


In [42]:
comments_res = get_comments("1lhoxch")
len(comments_res)

-- Query: https://oauth.reddit.com/comments/1lhoxch?limit=500&sort=old&depth=10
-- Query: https://oauth.reddit.com/api/morechildren?link_id=t3_1lhoxch&limit_children=false&sort=old&api_type=json&children=mz6qy4g,mz6d807,mz5zqk1,mz6cdqz,mz5wfnn,mz69drd,mz69lp2,mz64xcx,mz6e7iq,mz6fw11,mz7jjes,mz6ejnf,mz6kckn,mz728ff,mz5xoyd,mz5wkj9,mz6dq2d,mz5znhy,mz5zz6f,mz63n73,mz673og,mz6oo8i,mz7193w,mzald9x,mz7b9do,mz5ze2p,mz6x6w3,mz6f0dx,mz9dmp2,mz7u315,mz6jtws,mz695is,mz9dt1n,mz69zkl,mz7gnfe,mz7kmph,mz6a4fi,mz6buml,mz61sqq,mz64rw4,mz6ccl9,mz6hqxt,mz6corx,mz6oz3k,mz84wxq,mz6qmca,mz5x90s,mz6k5qc,mz62ero,mz7w9j4,mz9iha3,mz6bf4z,mz94oip,mz6ux30,mz7dade,mz6r8ph,mz6i9e2,mz7jrbt,mza6ttr,mz5xpr5,mz61fm6,mz6er71,mz6zrw4,mz66hrr,mz6ryas,mz6smm5,mz7i7nu,mz89gxp,mz8xeum,mz6w37c,mz7fwbr,mz813hd,mz8dax8,mz66u1s,mz688v8,mz62bcf,mz7jss7,mz6szms,mz5wtts,mz5xhf1,mzabew1,mz6kdjo,mz6oawt,mz6p405,mz6wq9s,mz7ixe4,mz7sjh7,mz80tgm,mz9ayny,mz5whm6,mz61jqn,mz647yh,mz67uql,mz68mb6,mz6kj90,mz7s9d3,mz6d8js,mz6i1l3,mz6jnjj,mz6m

7509

{'message': 'Forbidden', 'error': 403}
